# Tutorial: Taming Big Data With Apache Spark and Python - Hands On!
## Exercise 7 - Super Hero Degrees of Separation

### Setup

FindSpark

This will circumvent many issues with your system finding spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz
!mv spark-2.4.5-bin-hadoop2.7 spark-2.4.5

In [ ]:
import os
# Install java
!apt-get update -qq
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null 

!pip install -q findspark
 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["SPARK_HOME"] = "/content/spark-2.4.5"
!java -version

openjdk version "1.8.0_342"
OpenJDK Runtime Environment (build 1.8.0_342-8u342-b07-0ubuntu1~18.04-b07)
OpenJDK 64-Bit Server VM (build 25.342-b07, mixed mode)


In [ ]:
!git clone https://github.com/bangkit-pambudi/resource-spark.git

Cloning into 'resource-spark'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 38 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (38/38), done.


In [ ]:
import findspark
findspark.init()

Load Libraries

In [ ]:
from pyspark import SparkConf, SparkContext

Set the file path

In [ ]:
data_folder = "/content/resource-spark/data/"

Create the Spark Context

In [ ]:
# configure your Spark context; master node is local machine
conf = SparkConf().setMaster("local").setAppName("DegreesOfSeparation")

# create a spark context object
sc = SparkContext(conf = conf)

Establish search parameters: startCharacterID and targetCharacterID.

In [ ]:
startCharacterID = 5306 #SpiderMan
targetCharacterID = 14 #ADAM 3,031 (who?)

Define and set the accumulator to 0.

In [ ]:
hitCounter = sc.accumulator(0)

### Define Functions

Convert each line of our input Rdd to a format for BFS. It will be a key-value pair, with the key being the heroID and the value being the connections, the distance and the colour. We also initialize our startCharacterID.

In [ ]:
def convertToBFS(line):
    fields = line.split()
    heroID = int(fields[0])
    connections = []
    for connection in fields[1:]:
        connections.append(int(connection))
    
    colour = 'WHITE' #unprocessed
    distance = 9999 #infinite distance
    
    if (heroID == startCharacterID):
        colour = 'GRAY'
        distance = 0
    
    return (heroID, (connections, distance, colour))

Bring in your data and convert to BFS.

In [ ]:
def createStartingRdd():
    file_to_open = data_folder + "marvel-graph.txt"
    inputFile = sc.textFile(file_to_open)
    return inputFile.map(convertToBFS)

Define the mapper.

In [ ]:
def bfsMap(node):
    characterID = node[0] #heroID
    data = node[1] #connections, distance, colour
    connections = data[0]
    distance = data[1]
    colour = data[2]
    
    results = []
    
    #if this node needs to be expanded...
    if (colour == 'GRAY'):
        for connection in connections:
            newCharacterID = connection
            newDistance = distance + 1
            newColour = 'GRAY'
            if (targetCharacterID == connection):
                hitCounter.add(1)
            
            newEntry = (newCharacterID, ([], newDistance, newColour))
            print(type(newEntry))
            results.append(newEntry)
        
        #Since we've processed this node, colour it black
        colour = 'BLACK'
        
    #Emit the input node so we don't lose it
    print(type(results))
    results.append((characterID, (connections, distance, colour)))
    return results

Define the reducer.

In [ ]:
def bfsReduce(data1, data2):
    edges1 = data1[0]
    edges2 = data2[0]
    distance1 = data1[1]
    distance2 = data2[1]
    colour1 = data1[2]
    colour2 = data2[2]
    
    distance = 9999
    colour = colour1
    edges = []
    
    #See if one is the orignal node with its connections.
    # If so preserve them.
    # .extend() adds multiple elements to end of a list
    if(len(edges1) > 0):
        edges.extend(edges1)
    if(len(edges2) > 0):
        edges.extend(edges2)
        
    # Preserve minimum distance
    if(distance1 < distance):
        distance = distance1
        
    if(distance2 < distance):
        distance = distance2
        
    # Preserve darkest colour
    if(colour1 == 'WHITE' and (colour2 == 'GRAY' or colour2 == 'BLACK')):
        colour = colour2
        
    if(colour1 == 'GRAY' and colour2 == 'BLACK'):
        colour = colour2
        
    if(colour2 == 'WHITE' and (colour1 == 'GRAY' or colour1 == 'BLACK')):
        colour = colour1
        
    if(colour2 == 'GRAY' and colour1 == 'BLACK'):
        colour = colour1
    
    return(edges, distance, colour)

### Main Program

In [ ]:
iterationRdd = createStartingRdd()

We are assuming that degrees of separation will not exceed 10. We use flatMap(bfsMap) to find GREY nodes, expand them and then colour them BLACK. Map functions are transforms, due to lazy processing they do not cause anything to happen. So we call an action (i.e., count) to force it to get evaluated.

In [ ]:
for iteration in range(0, 10):
    print("Running BFS iteration# " + str(iteration+1))
    
    # Create new vertices as needed to darken or reduce distances in the 
    # reduce stage. If we encounter the node we're looking for as a GRAY
    # node, increment our accumulator to signal that we're done.
    mapped = iterationRdd.flatMap(bfsMap)
    
    # Note that mapped.count() action here forces the RDD to be evaluated, and
    # that's the only reason our accumulator is actually updated.
    print("Processing " + str(mapped.count()) + " values.")
    
    if (hitCounter.value > 0):
        print("Hit the target character! From " + str(hitCounter.value) \
              + " different direction(s).")
        break
        
    # Reducer combines data for each character ID, preserving the darkest
    # colour and shortest path.
    iterationRdd = mapped.reduceByKey(bfsReduce)

Running BFS iteration# 1
Processing 8330 values.
Running BFS iteration# 2
Processing 220615 values.
Hit the target character! From 1 different direction(s).
